In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.optimize import fsolve
import itertools

from f_detrend import detrend_fuction

sn.set_style('whitegrid')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Expanded and modified version (first column now string)
def read_table(filename,keyword='ciclo',rescale=1,ncols=5,start=2,end_row='Total',
               endword=None,omit_list=['t20_21'],index_to_int=True,debug=False):
    t0 = start
    t1 = ncols+start
    results_follow = 0
    row_names = []
    f = open(filename,'r')
    for line in f:
        words = line.split()
        if debug:
            print(words)
        if len(words)>1 and words[0] == keyword:
            results_follow=1
            if endword==None:
                colnames = words[t0:t1]
            else:
                buff_v = []
                for w in words[t0:]:
                    if w==endword:
                        break
                    else:
                        buff_v.append(w)
                colnames = buff_v[:-1]
                t1 = len(buff_v)+t0-1
            table_raw = np.empty(t1-t0)
        elif len(words)>1 and words[0] == end_row:
            break
        elif len(words)>1 and results_follow==1 and words[0] not in omit_list:
            row_names.append(words[0])
            buff_v = []
            for w in words[t0:t1]: 
                w = w.replace(",","")
                buff_v.append(w)
            table_raw = np.vstack((table_raw,np.array(buff_v,dtype=float)/rescale))
    table_raw = table_raw[1:,:]
    if debug:
        print(table_raw.shape)
        print(colnames)
    if index_to_int:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names,dtype=int))
    else:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names))
    
    
def file_split(filename,path,startwords=['Summary',],endword="Total",header=None):
    f = open(path+filename,'r')
    parent_file = filename[:filename.find('.')]
    di = 1
    writing = 0
    for line in f:
        words = line.split()
        if len(words)>0 and words[0] in startwords:
            sf = open(path+parent_file+'_{}.log'.format(di), "w")
            writing = 1
            if header!=None:
                sf.write(header)
        elif len(words)>0 and words[0]==endword and writing:
            sf.write(line)
            writing = 0
            sf.close()
            di+=1
        if writing:
            sf.write(line)
            
def normalise_table(table):
    cols = table.columns
    table['Total'] = table.sum(axis=1).copy()
    for col in cols:
        table[col] = table[col]/table['Total']
    return table   

In [3]:
dates = []
for year in range(1987,2022):
    for quarter in range(1,5):
        dates.append(str(year)+"Q"+str(quarter))
dates = dates[1:]

t0 = 59
t05 = 71
T = len(dates)
date_dict = dict(zip(range(t0,T+t0),dates))
date_dict_reverse = dict(zip(dates,range(0,T)))
date_dict_reverse_ciclo = dict(zip(dates,range(t0,T+t0)))

### Step 1: Read the table

In [17]:
path = './results/'

#file_split('sqtreg_mothers_3035.log',path,startwords=['q25','q50','q75'],endword="_cons",
#           header="wife_ten_y | Coefficient std t p_stat lower_95 upper_95")

tab_moms_45_25 = read_table(path+'sqtreg_mothers_3035_1.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_moms_45_50 = read_table(path+'sqtreg_mothers_3035_2.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_moms_45_75 = read_table(path+'sqtreg_mothers_3035_3.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)

#file_split('sqtreg_mothers_0k_3035.log',path,startwords=['q25','q50','q75'],endword="_cons",
#           header="wife_ten_y | Coefficient std t p_stat lower_95 upper_95")

tab_moms_45_25_0k = read_table(path+'sqtreg_mothers_0k_3035_1.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_moms_45_50_0k = read_table(path+'sqtreg_mothers_0k_3035_2.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_moms_45_75_0k = read_table(path+'sqtreg_mothers_0k_3035_3.log',
                         keyword='wife_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)

Check the table values

In [18]:
tab_moms_45_25

,Coefficient,std,t,p_stat,lower_95,upper_95q25
hub_ten_y,0.176281,0.006263,28.15,0.000,0.164006,0.188556
hub_ten_y2,-0.006224,0.000309,-20.11,0.000,-0.006831,-0.005618
part_time,-0.596846,0.027871,-21.41,0.000,-0.651473,-0.542220
college,0.977132,0.034599,28.24,0.000,0.909319,1.044945
less_hs,-0.320664,0.053442,-6.00,0.000,-0.425411,-0.215917
hub_age,-0.016100,0.002957,-5.45,0.000,-0.021895,-0.010305
hub_se,0.034605,0.031713,1.09,0.275,-0.027554,0.096764
hub_college,0.375520,0.050317,7.46,0.000,0.276899,0.474142
hub_less_hs,-0.239290,0.057481,-4.16,0.000,-0.351953,-0.126626
t08_11,0.334483,0.039265,8.52,0.000,0.257523,0.411442


Select coefficients of interest

In [24]:
time_periods_thin = ['t08_11', 't11_14','t14_17', 't17_20', 't20', 't21', 't22']
time_periods_thin_labels = ['2008-2011','2011-2014','2014-2017','2017-2020','2020','2021','2022']
columns_of_interest = ['Coefficient','std']

tab_moms_45_25_t = tab_moms_45_25.loc[time_periods_thin,columns_of_interest]
tab_moms_45_25_t

,Coefficient,std
t08_11,0.334483,0.039265
t11_14,1.019554,0.056596
t14_17,1.034728,0.070264
t17_20,0.226864,0.043074
t20,0.244755,0.084752
t21,0.024253,0.061547
t22,-0.002503,0.068778


Rename and round

In [25]:
tab_moms_45_25_t.rename(index = dict(zip(time_periods_thin,time_periods_thin_labels)), inplace=True)
tab_moms_45_25_t = np.round(tab_moms_45_25_t,3)
tab_moms_45_25_t

,Coefficient,std
2008-2011,0.334,0.039
2011-2014,1.020,0.057
2014-2017,1.035,0.070
2017-2020,0.227,0.043
2020,0.245,0.085
2021,0.024,0.062
2022,-0.003,0.069


Repeat for all

In [34]:
datasets = [tab_moms_45_25_0k.copy(), tab_moms_45_50_0k.copy(), tab_moms_45_75_0k.copy()]

for i,data in enumerate(datasets):
    data = data.loc[time_periods_thin,columns_of_interest]
    data.rename(index = dict(zip(time_periods_thin,time_periods_thin_labels)), inplace=True)
    data = np.round(data,3)
    datasets[i] = data

### Step 2: Build table of tables

Start with the first row

In [41]:
period_0 = time_periods_thin_labels[0]
export_df = pd.DataFrame({period_0:[datasets[i].loc[period_0,'Coefficient'] for i in range(len(datasets))]},index=['p25','p50','p75'])
export_df

,2008-2011
p25,0.184
p50,-0.062
p75,-0.148


Then add std and format

In [46]:
export_df[str(period_0+" std")] = [datasets[i].loc[period_0,'std'] for i in range(len(datasets))]
export_df[str(period_0+" std")] = export_df[str(period_0+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")
export_df

,2008-2011,2008-2011 std
p25,0.184,(0.04)
p50,-0.062,(0.07)
p75,-0.148,(0.10)


Automate

In [50]:
for period in time_periods_thin_labels[1:]:
    export_df[period] = [datasets[i].loc[period,'Coefficient'] for i in range(len(datasets))]
    export_df[str(period+" std")] = [datasets[i].loc[period,'std'] for i in range(len(datasets))]
    export_df[str(period+" std")] = export_df[str(period+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")

export_df.T

,p25,p50,p75
2008-2011,0.184,-0.062,-0.148
2008-2011 std,(0.04),(0.07),(0.10)
2011-2014,0.596,0.655,0.336
2011-2014 std,(0.08),(0.09),(0.10)
2014-2017,0.243,0.834,0.813
2014-2017 std,(0.06),(0.12),(0.12)
2017-2020,-0.075,-0.125,0.492
2017-2020 std,(0.06),(0.09),(0.13)
2020,-0.252,-0.739,-0.778
2020 std,(0.05),(0.13),(0.25)


Erase the std columns and pivot

In [52]:
export_df = export_df.T
for period in time_periods_thin_labels:
    export_df.rename({str(period)+" std":""}, inplace=True)

export_df

### Step 3: Export

In [54]:
print(export_df.to_latex())

\begin{tabular}{llll}
\toprule
{} &     p25 &     p50 &     p75 \\
\midrule
2008-2011 &   0.184 &  -0.062 &  -0.148 \\
          &  (0.04) &  (0.07) &  (0.10) \\
2011-2014 &   0.596 &   0.655 &   0.336 \\
          &  (0.08) &  (0.09) &  (0.10) \\
2014-2017 &   0.243 &   0.834 &   0.813 \\
          &  (0.06) &  (0.12) &  (0.12) \\
2017-2020 &  -0.075 &  -0.125 &   0.492 \\
          &  (0.06) &  (0.09) &  (0.13) \\
2020      &  -0.252 &  -0.739 &  -0.778 \\
          &  (0.05) &  (0.13) &  (0.25) \\
2021      &  -0.011 &  -0.276 &   0.051 \\
          &  (0.05) &  (0.12) &  (0.23) \\
2022      &   -0.16 &    -0.3 &  -0.567 \\
          &  (0.13) &  (0.14) &  (0.14) \\
\bottomrule
\end{tabular}



C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_2272\1701916704.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(export_df.to_latex())


To a file

In [55]:
export_df.to_latex('./regtabs/formatted/trial.tex')

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_2272\826207401.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  export_df.to_latex('./regtabs/formatted/trial.tex')


### Step 4: Adding another table

Let's add the moms! All the steps together now

In [57]:
datasets = [tab_moms_45_25.copy(), tab_moms_45_50.copy(), tab_moms_45_75.copy()]

for i,data in enumerate(datasets):
    data = data.loc[time_periods_thin,columns_of_interest]
    data.rename(index = dict(zip(time_periods_thin,time_periods_thin_labels)), inplace=True)
    data = np.round(data,3)
    datasets[i] = data
    
period_0 = time_periods_thin_labels[0]
export_df_moms = pd.DataFrame({period_0:[datasets[i].loc[period_0,'Coefficient'] for i in range(len(datasets))]},index=['p25','p50','p75'])
export_df_moms[str(period_0+" std")] = [datasets[i].loc[period_0,'std'] for i in range(len(datasets))]
export_df_moms[str(period_0+" std")] = export_df_moms[str(period_0+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")
for period in time_periods_thin_labels[1:]:
    export_df_moms[period] = [datasets[i].loc[period,'Coefficient'] for i in range(len(datasets))]
    export_df_moms[str(period+" std")] = [datasets[i].loc[period,'std'] for i in range(len(datasets))]
    export_df_moms[str(period+" std")] = export_df_moms[str(period+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")
    
export_df_moms = export_df_moms.T
for period in time_periods_thin_labels:
    export_df_moms.rename({str(period)+" std":""}, inplace=True)

Join using ```pd.concat(axis=1)```

In [65]:
export_women = pd.concat([export_df_moms,export_df], keys=['With kids', 'No Kids'], axis=1)
export_women

With kids                 No Kids                
                p25     p50     p75     p25     p50     p75
2008-2011     0.334   0.238   0.358   0.184  -0.062  -0.148
             (0.04)  (0.07)  (0.09)  (0.04)  (0.07)  (0.10)
2011-2014      1.02   1.115   0.845   0.596   0.655   0.336
             (0.06)  (0.06)  (0.09)  (0.08)  (0.09)  (0.10)
2014-2017     1.035   1.875   1.406   0.243   0.834   0.813
             (0.07)  (0.09)  (0.08)  (0.06)  (0.12)  (0.12)
2017-2020     0.227   0.957   1.535  -0.075  -0.125   0.492
             (0.04)  (0.08)  (0.10)  (0.06)  (0.09)  (0.13)
2020          0.245   0.243   1.478  -0.252  -0.739  -0.778
             (0.09)  (0.15)  (0.17)  (0.05)  (0.13)  (0.25)
2021          0.024  -0.103   0.801  -0.011  -0.276   0.051
             (0.06)  (0.13)  (0.18)  (0.05)  (0.12)  (0.23)
2022         -0.003  -0.616   -0.75   -0.16    -0.3  -0.567
             (0.07)  (0.15)  (0.31)  (0.13)  (0.14)  (0.14)

### Step 5: add final layer (men)

In [62]:
path = './results/'

#file_split('sqtreg_fathers_3035.log',path,startwords=['q25','q50','q75'],endword="_cons",
#           header="hub_ten_y | Coefficient std t p_stat lower_95 upper_95")

tab_dads_45_25 = read_table(path+'sqtreg_fathers_3035_1.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_dads_45_50 = read_table(path+'sqtreg_fathers_3035_2.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_dads_45_75 = read_table(path+'sqtreg_fathers_3035_3.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)

#file_split('sqtreg_fathers_0k_3035.log',path,startwords=['q25','q50','q75'],endword="_cons",
#           header="hub_ten_y | Coefficient std t p_stat lower_95 upper_95")

tab_dads_45_25_0k = read_table(path+'sqtreg_fathers_0k_3035_1.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_dads_45_50_0k = read_table(path+'sqtreg_fathers_0k_3035_2.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)
tab_dads_45_75_0k = read_table(path+'sqtreg_fathers_0k_3035_3.log',
                         keyword='hub_ten_y',rescale=1,ncols=6,start=2,omit_list=['age2','period_y'],index_to_int=False)

Packed up function

In [61]:
def format_reg_table(datasets):
    for i,data in enumerate(datasets):
        data = data.loc[time_periods_thin,columns_of_interest]
        data.rename(index = dict(zip(time_periods_thin,time_periods_thin_labels)), inplace=True)
        data = np.round(data,3)
        datasets[i] = data

    period_0 = time_periods_thin_labels[0]
    export_df_moms = pd.DataFrame({period_0:[datasets[i].loc[period_0,'Coefficient'] for i in range(len(datasets))]},index=['p25','p50','p75'])
    export_df_moms[str(period_0+" std")] = [datasets[i].loc[period_0,'std'] for i in range(len(datasets))]
    export_df_moms[str(period_0+" std")] = export_df_moms[str(period_0+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")
    for period in time_periods_thin_labels[1:]:
        export_df_moms[period] = [datasets[i].loc[period,'Coefficient'] for i in range(len(datasets))]
        export_df_moms[str(period+" std")] = [datasets[i].loc[period,'std'] for i in range(len(datasets))]
        export_df_moms[str(period+" std")] = export_df_moms[str(period+" std")].apply(lambda x: "("+"{:.2f}".format(float(x))+")")

    export_df_moms = export_df_moms.T
    for period in time_periods_thin_labels:
        export_df_moms.rename({str(period)+" std":""}, inplace=True)
    return export_df_moms

Format dads

In [63]:
datasets_dads = [tab_dads_45_25.copy(), tab_dads_45_50.copy(), tab_dads_45_75.copy()]
datasets_no_dads = [tab_dads_45_25_0k.copy(), tab_dads_45_50_0k.copy(), tab_dads_45_75_0k.copy()]

formatted_table_dads = format_reg_table(datasets_dads)
formatted_table_no_dads = format_reg_table(datasets_no_dads)

Stack

In [67]:
export_men = pd.concat([formatted_table_dads,formatted_table_no_dads], keys=['With kids', 'No Kids'], axis=1)

export_all = pd.concat([export_women,export_men], keys=['Women', 'Men'], axis=1)
export_all

Women                                               Men          \
          With kids                 No Kids                 With kids           
                p25     p50     p75     p25     p50     p75       p25     p50   
2008-2011     0.334   0.238   0.358   0.184  -0.062  -0.148     0.105   0.203   
             (0.04)  (0.07)  (0.09)  (0.04)  (0.07)  (0.10)    (0.05)  (0.10)   
2011-2014      1.02   1.115   0.845   0.596   0.655   0.336     0.591     0.9   
             (0.06)  (0.06)  (0.09)  (0.08)  (0.09)  (0.10)    (0.07)  (0.11)   
2014-2017     1.035   1.875   1.406   0.243   0.834   0.813     0.285   0.841   
             (0.07)  (0.09)  (0.08)  (0.06)  (0.12)  (0.12)    (0.08)  (0.09)   
2017-2020     0.227   0.957   1.535  -0.075  -0.125   0.492    -0.123    0.23   
             (0.04)  (0.08)  (0.10)  (0.06)  (0.09)  (0.13)    (0.07)  (0.13)   
2020          0.245   0.243   1.478  -0.252  -0.739  -0.778      0.13   0.047   
             (0.09)  (0.15)  (0.17)  (0.05)  (0.13)  (0.25)    (0.13)  (0.17)   
2021          0.024  -0.103   0.801  -0.011  -0.276   0.051      0.22    0.21   
             (0.06)  (0.13)  (0.18)  (0.05)  (0.12)  (0.23)    (0.22)  (0.25)   
2022         -0.003  -0.616   -0.75   -0.16    -0.3  -0.567    -0.108  -0.189   
             (0.07)  (0.15)  (0.31)  (0.13)  (0.14)  (0.14)    (0.18)  (0.14)   

                                           
                  No Kids                  
              p75     p25     p50     p75  
2008-2011   0.295   0.246  -0.155  -0.102  
           (0.10)  (0.06)  (0.07)  (0.09)  
2011-2014   0.597     0.6   0.487   0.429  
           (0.14)  (0.07)  (0.09)  (0.11)  
2014-2017   0.728  -0.047   0.359   0.627  
           (0.12)  (0.09)  (0.13)  (0.14)  
2017-2020   1.137  -0.219   -0.43   0.702  
           (0.15)  (0.05)  (0.09)  (0.13)  
2020        0.783  -0.093  -0.611  -0.305  
           (0.22)  (0.10)  (0.18)  (0.24)  
2021        1.436  -0.064   -0.42  -0.119  
           (0.30)  (0.11)  (0.17)  (0.27)  
2022       -0.142   0.006   0.112  -0.295  
           (0.20)  (0.15)  (0.24)  (0.29)

Export

In [68]:
export_all.to_latex('./regtabs/formatted/reg_tabs_3035.tex')

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_2272\1734908947.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  export_all.to_latex('./regtabs/formatted/reg_tabs_3035.tex')
